# **Data cleaning notebook**

## Objectives

* Evaluate missing data
* Clean data

## Inputs

* outputs/datasets/collection/hotel_bookings.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned


---

# Change working directory

Change the working directory from its current folder to its parent folder
* Access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

Make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load collected data

In [ ]:
import pandas as pd

df_raw_path = "outputs/datasets/collection/hotel_bookings.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

# Data exploration

Check the distribution and shape of a variable with missing data.

In [ ]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

In [ ]:
from ydata_profiling import ProfileReport

if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

# Correlation and PPS analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(
            df,
            annot=True,
            xticklabels=True,
            yticklabels=True,
            mask=mask, cmap='viridis',
            annot_kws={"size": font_annot},
            ax=axes,
            linewidth=0.5
        )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(
            df,
            annot=True,
            xticklabels=True,
            yticklabels=True,
            mask=mask, cmap='rocket_r',
            annot_kws={"size": font_annot},
            linewidth=0.05, linecolor='grey'
        )
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(
        columns='x', index='y', values='ppscore'
    )

    pps_score_stats = pps_matrix_raw.query(
        "ppscore < 1"
    ).filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(
    df_corr_pearson,
    df_corr_spearman,
    pps_matrix,
    CorrThreshold,
    PPS_Threshold,
    figsize=(20, 12),
    font_annot=8
):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(
        df=df_corr_spearman,
        threshold=CorrThreshold,
        figsize=figsize,
        font_annot=font_annot
    )

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(
        df=df_corr_pearson,
        threshold=CorrThreshold,
        figsize=figsize,
        font_annot=font_annot
    )

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(
        f"PPS detects linear or non-linear relationships between two columns.\n"
        f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n"
    )
    heatmap_pps(
        df=pps_matrix,
        threshold=PPS_Threshold,
        figsize=figsize,
        font_annot=font_annot
    )

Calculate Correlations and Power Predictive Score.

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

Display heatmaps.

In [ ]:
DisplayCorrAndPPS(
    df_corr_pearson = df_corr_pearson,
    df_corr_spearman = df_corr_spearman, 
    pps_matrix = pps_matrix,
    CorrThreshold = 0.4,
    PPS_Threshold =0.2,
    figsize=(12,10),
    font_annot=10
)

# Data cleaning

## Assess missing data levels

Custom function to display missing data levels in a DataFrame, it shows the absolute levels, relative levels, and data type.

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (
        pd.DataFrame(
            data={
                "RowsWithMissingData": missing_data_absolute,
                "PercentageOfDataset": missing_data_percentage,
                "DataType": df.dtypes
            }
        ).sort_values(
            by=['PercentageOfDataset'],
            ascending=False
        ).query("PercentageOfDataset > 0")
    )

    return df_missing_data

Check missing data levels for the collected dataset.

In [ ]:
EvaluateMissingData(df)

## Data cleaning summary

- Drop:
    - `company`, irrelevant feature as it's company ID number.
    - `agent`, irrelevant feature as it's agent ID number.
    - `country`, noisy data due to high cardinality.

## Split Train and Test set

In [ ]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet, _, __ = train_test_split(
    df,
    df['is_canceled'],
    test_size=0.2,
    random_state=0
)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

In [ ]:
df_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {df_missing_data.shape[0]} variables with missing data \n")
df_missing_data

## Drop variables

- Drop variables by imputation.
- Select variables to apply the imputation approach.

In [ ]:
variables_method = ['company', 'agent', 'country']

print(f"* {len(variables_method)} variables to drop \n\n"
    f"{variables_method}")

Create a separate DataFrame applying this imputation approach to the selected variables.

In [ ]:
from feature_engine.selection import DropFeatures

imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)
df_method = imputer.transform(TrainSet)

Assess the effect on the variable's distribution.

- There is no effect on the distribution of the variable since only columns are removed, not rows.

Apply transformation to data.

In [ ]:
from feature_engine.selection import DropFeatures

imputer = DropFeatures(features_to_drop=variables_method)
imputer.fit(TrainSet)

TrainSet, TestSet = imputer.transform(TrainSet), imputer.transform(TestSet)

Evaluate missing data levels again to double check after imputation.

In [ ]:
EvaluateMissingData(TrainSet)

In [ ]:
EvaluateMissingData(TestSet)

---

# Push clean data to repo

In [ ]:
import os

try:
    # Create outputs/datasets/collection folder
    os.makedirs(name='outputs/datasets/cleaned')
except Exception as e:
    print(e)

## Train set

In [ ]:
TrainSet.to_csv(
    "outputs/datasets/cleaned/train_set_cleaned.csv",
    index=False
)

## Test set

In [ ]:
TestSet.to_csv(
    "outputs/datasets/cleaned/test_set_cleaned.csv",
    index=False
)

---

# Next steps

Carry out feature engineering in preparation for modelling.